In [1]:
#Import Dependencies 
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
#Import CSV
# citibike = "../ETL/CitiBike.csv"
citibike = pd.read_csv("CitiBike.csv")
citibike.head()

,Date,Trips over the past 24-hours (midnight to 11:59pm),Cumulative trips (since launch):,Miles traveled today (midnight to 11:59 pm),Miles traveled to date:,Total Annual Members,Annual Member Sign-Ups (midnight to 11:59 pm),24-Hour Passes Purchased (midnight to 11:59 pm),7-Day Passes Purchased (midnight to 11:59 pm),3-Day Passes Purchased (midnight to 11:59 pm)
0,5/27/2013,9767,9767.0,21533,21533.0,17216.0,2043.0,0.0,0.0,NaN
1,5/28/2013,5215,14982.0,8780,30313.0,19816.0,2598.0,0.0,0.0,NaN
2,5/29/2013,10981,25963.0,21898,52211.0,21986.0,2167.0,0.0,0.0,NaN
3,5/30/2013,9850,35813.0,20321,72532.0,23985.0,1998.0,0.0,0.0,NaN
4,5/31/2013,9253,45066.0,20243,92775.0,25615.0,1630.0,0.0,0.0,NaN


In [3]:
#Cleaned bike data for easier data retrieval. Names were too long and were shortened for simple queries.

#Manipulated date format for ease of use
citibike_df = citibike.assign(dateint = citibike['Date'].str.replace("/", "").astype(int))


# Bike data columns renamed for simple queries.
citibike_cleaned1 = citibike_df.rename(columns = {
  'Trips over the past 24-hours (midnight to 11:59pm)': 'daily_trips',
  'Cumulative trips (since launch):': 'cumulative_trips',
  'Miles traveled today (midnight to 11:59 pm)': 'daily_miles',
  'Miles traveled to date:': 'cumulative_miles',
  'Total Annual Members': 'cumulative_members'
})

# Dropped columns that were unnecessary for our data exploration.

citibike_cleaned2 = citibike_cleaned1.drop(['Date',
  'Annual Member Sign-Ups (midnight to 11:59 pm)',
  '24-Hour Passes Purchased (midnight to 11:59 pm)',
  '7-Day Passes Purchased (midnight to 11:59 pm)',
  '3-Day Passes Purchased (midnight to 11:59 pm)'
], axis = 1)


#reordering the df for easy of use in sql
citibike_df = citibike_cleaned2
citibike_df = citibike_df[['dateint', 'daily_trips', 'cumulative_trips', 'daily_miles', 'cumulative_miles', 'cumulative_members']]

citibike_df.head()

,dateint,daily_trips,cumulative_trips,daily_miles,cumulative_miles,cumulative_members
0,5272013,9767,9767.0,21533,21533.0,17216.0
1,5282013,5215,14982.0,8780,30313.0,19816.0
2,5292013,10981,25963.0,21898,52211.0,21986.0
3,5302013,9850,35813.0,20321,72532.0,23985.0
4,5312013,9253,45066.0,20243,92775.0,25615.0


In [4]:
weather = pd.read_csv('1784274.csv')
weather.head()

,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09
0,USW00094728,"NY CITY CENTRAL PARK, NY US",5/27/2013,3.80,0.00,0.0,0.0,NaN,73.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00094728,"NY CITY CENTRAL PARK, NY US",5/28/2013,2.68,0.52,0.0,0.0,NaN,68.0,56.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00094728,"NY CITY CENTRAL PARK, NY US",5/29/2013,1.57,0.00,0.0,0.0,NaN,82.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00094728,"NY CITY CENTRAL PARK, NY US",5/30/2013,5.14,0.00,0.0,0.0,NaN,90.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,USW00094728,"NY CITY CENTRAL PARK, NY US",5/31/2013,4.03,0.00,0.0,0.0,NaN,90.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Grouped csv data by date in order to preserve hierarchy amongst the scattered weather data. 
weather = weather.assign(dateint = weather['DATE'].str.replace("/", "").astype(int))
weather = weather.groupby(['DATE']).mean()
weather = weather.reset_index()
filterweather = weather[['DATEINT', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN']]


#Renamed columns for ease of use in sql
filterweather = filterweather.rename(columns = {
  'DATEINT': 'dateint',
  'PRCP': 'percip_in_inches',
  'SNOW': 'snow_in_inches',
  'SNWD': 'snow_depth_in_inches',
  'TMAX': 'max_temp',
  'TMIN': 'min_temp'
})
filterweather.head()



In [6]:
connection_string = "postgres:postgres@localhost:5432/ETL_citibike"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
citibike_df.to_sql(name='citibike', con=engine, if_exists='append', index=True)

In [ ]:
filterweather.to_sql(name='weather', con=engine, if_exists='append', index=True)